## Jakob Ohmayer (4742300)

In [2]:
import os
import zipfile
import torch

import requests
from torch.utils.data import DataLoader
from torchvision import transforms as T
from torchvision.datasets import CIFAR10
from tqdm import tqdm
import torch.nn as nn

### Load model ResNet34

In [3]:
import adapt

In [4]:
from models.resnet import resnet34

In [5]:
threads = 40
torch.set_num_threads(threads)

#maybe better performance
%env OMP_PLACES=cores
%env OMP_PROC_BIND=close
%env OMP_WAIT_POLICY=active

env: OMP_PLACES=cores
env: OMP_PROC_BIND=close
env: OMP_WAIT_POLICY=active


### Choose approximate multiplier 

Here the multiplier SPR12_44 is used.

In [6]:
axx_mult = 'SPR12_44'

### Load model for evaluation

In [7]:
model = resnet34(pretrained=True, axx_mult = axx_mult)

model.eval() # for evaluation

Using /root/.cache/torch_extensions as PyTorch extensions root...
Emitting ninja build file /root/.cache/torch_extensions/PyInit_conv2d_SPR12_44/build.ninja...
Building extension module PyInit_conv2d_SPR12_44...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module PyInit_conv2d_SPR12_44...
Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module PyInit_conv2d_SPR12_44, skipping build step...
Loading extension module PyInit_conv2d_SPR12_44...
Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module PyInit_conv2d_SPR12_44, skipping build step...
Loading extension module PyInit_conv2d_SPR12_44...
Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module PyInit_conv2d_SPR12_44, skipping bui

ResNet(
  (conv1): AdaPT_Conv2d(
    3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
    (quantizer): TensorQuantizer(8bit per-tensor amax=dynamic calibrator=HistogramCalibrator quant)
    (quantizer_w): TensorQuantizer(8bit per-tensor amax=dynamic calibrator=HistogramCalibrator quant)
  )
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): AdaPT_Conv2d(
        64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (quantizer): TensorQuantizer(8bit per-tensor amax=dynamic calibrator=HistogramCalibrator quant)
        (quantizer_w): TensorQuantizer(8bit per-tensor amax=dynamic calibrator=HistogramCalibrator quant)
      )
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(

### Load dataset


In [8]:
def val_dataloader(mean = (0.4914, 0.4822, 0.4465), std = (0.2471, 0.2435, 0.2616)):

    transform = T.Compose(
        [
            T.ToTensor(),
            T.Normalize(mean, std),
        ]
    )
    dataset = CIFAR10(root="datasets/cifar10_data", train=False, download=True, transform=transform)
    dataloader = DataLoader(
        dataset,
        batch_size=128,
        num_workers=0,
        drop_last=True,
        pin_memory=False,
    )
    return dataloader

transform = T.Compose(
        [
            T.RandomCrop(32, padding=4),
            T.RandomHorizontalFlip(),
            T.ToTensor(),
            T.Normalize(mean = (0.4914, 0.4822, 0.4465), std = (0.2471, 0.2435, 0.2616)),
        ]
    )
dataset = CIFAR10(root="datasets/cifar10_data", train=True, download=True, transform=transform)

evens = list(range(0, len(dataset), 10))
trainset_1 = torch.utils.data.Subset(dataset, evens)

data = val_dataloader()

# data_t is used for calibration purposes and is a subset of train-set
data_t = DataLoader(trainset_1, batch_size=128,
                                            shuffle=False, num_workers=0)


Files already downloaded and verified
Files already downloaded and verified


### Run model calibration for quantization

Calibrates the quantization parameters. Is rerun after each training epoch.

In [9]:
from pytorch_quantization import nn as quant_nn
from pytorch_quantization import calib

def collect_stats(model, data_loader, num_batches):
     """Feed data to the network and collect statistic"""

     # Enable calibrators
     for name, module in model.named_modules():
         if isinstance(module, quant_nn.TensorQuantizer):
             if module._calibrator is not None:
                 module.disable_quant()
                 module.enable_calib()
             else:
                 module.disable()

     for i, (image, _) in tqdm(enumerate(data_loader), total=num_batches):
         model(image.cpu())
         if i >= num_batches:
             break

     # Disable calibrators
     for name, module in model.named_modules():
         if isinstance(module, quant_nn.TensorQuantizer):
             if module._calibrator is not None:
                 module.enable_quant()
                 module.disable_calib()
             else:
                 module.enable()

def compute_amax(model, **kwargs):
 # Load calib result
 for name, module in model.named_modules():
     if isinstance(module, quant_nn.TensorQuantizer):
         if module._calibrator is not None:
             if isinstance(module._calibrator, calib.MaxCalibrator):
                 module.load_calib_amax()
             else:
                 module.load_calib_amax(**kwargs)
         print(F"{name:40}: {module}")
 model.cpu()

def calibrate_model(model, data_t):
    # It is a bit slow since we collect histograms on CPU
    with torch.no_grad():
        stats = collect_stats(model, data_t, num_batches=2)
        amax = compute_amax(model, method="percentile", percentile=99.99)
    
        # optional - test different calibration methods
        #amax = compute_amax(model, method="mse")
        #amax = compute_amax(model, method="entropy")


In [10]:
calibrate_model(model, data_t)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:17<00:00,  8.92s/it]
W1117 21:51:50.554443 134165911848768 tensor_quantizer.py:173] Disable HistogramCalibrator
W1117 21:51:50.557247 134165911848768 tensor_quantizer.py:173] Disable HistogramCalibrator
W1117 21:51:50.557740 134165911848768 tensor_quantizer.py:173] Disable HistogramCalibrator
W1117 21:51:50.558352 134165911848768 tensor_quantizer.py:173] Disable HistogramCalibrator
W1117 21:51:50.559018 134165911848768 tensor_quantizer.py:173] Disable HistogramCalibrator
W1117 21:51:50.559560 134165911848768 tensor_quantizer.py:173] Disable HistogramCalibrator
W1117 21:51:50.560225 134165911848768 tensor_quantizer.py:173] Disable HistogramCalibrator
W1117 21:51:50.560859 134165911848768 tensor_quantizer.py:173] Disable HistogramCalibrator
W1117 21:51:50.561560 134165911848768 tensor_quantizer.py:173] Disable HistogramCalibrato

W1117 21:51:50.603738 134165911848768 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W1117 21:51:50.604362 134165911848768 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W1117 21:51:50.605020 134165911848768 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W1117 21:51:50.605676 134165911848768 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W1117 21:51:50.606353 134165911848768 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W1117 21:51:50.606976 134165911848768 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W1117 21:51:50.607666 134165911848768 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W1117 21:51:50.608319 134165911848768 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W1117 21:51:50.609015 134165911848768 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W1117 21:51:50.609676 134165911848768

conv1.quantizer                         : TensorQuantizer(8bit per-tensor amax=2.1255 calibrator=HistogramCalibrator quant)
conv1.quantizer_w                       : TensorQuantizer(8bit per-tensor amax=0.1418 calibrator=HistogramCalibrator quant)
layer1.0.conv1.quantizer                : TensorQuantizer(8bit per-tensor amax=0.6598 calibrator=HistogramCalibrator quant)
layer1.0.conv1.quantizer_w              : TensorQuantizer(8bit per-tensor amax=0.0555 calibrator=HistogramCalibrator quant)
layer1.0.conv2.quantizer                : TensorQuantizer(8bit per-tensor amax=0.3260 calibrator=HistogramCalibrator quant)
layer1.0.conv2.quantizer_w              : TensorQuantizer(8bit per-tensor amax=0.0327 calibrator=HistogramCalibrator quant)
layer1.1.conv1.quantizer                : TensorQuantizer(8bit per-tensor amax=0.5872 calibrator=HistogramCalibrator quant)
layer1.1.conv1.quantizer_w              : TensorQuantizer(8bit per-tensor amax=0.0395 calibrator=HistogramCalibrator quant)
layer1.1

### Run model evaluation (before re-training)

In [11]:
import timeit
correct = 0
total = 0

model.eval()
start_time = timeit.default_timer()
with torch.no_grad():
    for iteraction, (images, labels) in tqdm(enumerate(data), total=len(data)):
        images, labels = images.to("cpu"), labels.to("cpu")
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(timeit.default_timer() - start_time)
print('Accuracy of the network on the 10000 test images: %.4f %%' % (
    100 * correct / total))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 78/78 [29:07<00:00, 22.41s/it]

1747.6586084780001
Accuracy of the network on the 10000 test images: 33.7740 %


**Accuracy before re-training**: 33.77%

### Run approximate-aware re-training for 15 epochs


As re-training takes quite long and brakes sometimes the individual models are saved after each epoch to enable loading these models to continue fine-tuning. Therefore the log output of the following cell is not correct. The majority of the output is included in the file ``training.log`` in the same folder

In [ ]:
from adapt.references.classification.train import evaluate, train_one_epoch, load_data

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

EPOCHS = 15

# load model if necessary
load_epoch = 2
model.load_state_dict(torch.load(f"./saved/retrained_model_epoch_{load_epoch}.pth"))
calibrate_model(model, data_t)

# finetune the model for one epoch based on data_t subset
for epoch in range(EPOCHS):
    EPOCH_NUM = epoch + 1
    if EPOCH_NUM > load_epoch:
        print(f"Epoch {epoch + 1}/{EPOCHS}")
        train_one_epoch(model, criterion, optimizer, data_t, "cpu", EPOCH_NUM, 1)
        calibrate_model(model, data_t)

        # Save the model after each epoch
        model_path = f"./saved/retrained_model_epoch_{EPOCH_NUM}.pth"
        torch.save(model.state_dict(), model_path)
        print(f"Model saved to {model_path} after epoch {EPOCH_NUM}")
    else:
        print(f"Skipping epoch {EPOCH_NUM} as pretrained model exists")

W1117 21:51:51.078365 134165911848768 tensor_quantizer.py:402] conv1.quantizer: Overwriting amax.
W1117 21:51:51.079574 134165911848768 tensor_quantizer.py:402] conv1.quantizer_w: Overwriting amax.
W1117 21:51:51.089933 134165911848768 tensor_quantizer.py:402] layer1.0.conv1.quantizer: Overwriting amax.
W1117 21:51:51.090771 134165911848768 tensor_quantizer.py:402] layer1.0.conv1.quantizer_w: Overwriting amax.
W1117 21:51:51.093954 134165911848768 tensor_quantizer.py:402] layer1.0.conv2.quantizer: Overwriting amax.
W1117 21:51:51.094571 134165911848768 tensor_quantizer.py:402] layer1.0.conv2.quantizer_w: Overwriting amax.
W1117 21:51:51.096243 134165911848768 tensor_quantizer.py:402] layer1.1.conv1.quantizer: Overwriting amax.
W1117 21:51:51.096866 134165911848768 tensor_quantizer.py:402] layer1.1.conv1.quantizer_w: Overwriting amax.
W1117 21:51:51.098430 134165911848768 tensor_quantizer.py:402] layer1.1.conv2.quantizer: Overwriting amax.
W1117 21:51:51.099478 134165911848768 tensor_qu

W1117 21:53:10.793441 134165911848768 tensor_quantizer.py:173] Disable HistogramCalibrator
W1117 21:53:10.794545 134165911848768 tensor_quantizer.py:173] Disable HistogramCalibrator
W1117 21:53:10.795618 134165911848768 tensor_quantizer.py:173] Disable HistogramCalibrator
W1117 21:53:10.796675 134165911848768 tensor_quantizer.py:173] Disable HistogramCalibrator
W1117 21:53:10.797763 134165911848768 tensor_quantizer.py:173] Disable HistogramCalibrator
W1117 21:53:10.798811 134165911848768 tensor_quantizer.py:173] Disable HistogramCalibrator
W1117 21:53:10.799889 134165911848768 tensor_quantizer.py:173] Disable HistogramCalibrator
W1117 21:53:10.800963 134165911848768 tensor_quantizer.py:173] Disable HistogramCalibrator
W1117 21:53:10.802589 134165911848768 tensor_quantizer.py:173] Disable HistogramCalibrator
W1117 21:53:10.803675 134165911848768 tensor_quantizer.py:173] Disable HistogramCalibrator
W1117 21:53:10.804737 134165911848768 tensor_quantizer.py:173] Disable HistogramCalibrator

W1117 21:53:10.874004 134165911848768 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W1117 21:53:10.874636 134165911848768 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W1117 21:53:10.875267 134165911848768 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W1117 21:53:10.875876 134165911848768 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W1117 21:53:10.876517 134165911848768 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W1117 21:53:10.877136 134165911848768 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W1117 21:53:10.877810 134165911848768 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W1117 21:53:10.878405 134165911848768 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W1117 21:53:10.879041 134165911848768 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W1117 21:53:10.879668 134165911848768

conv1.quantizer                         : TensorQuantizer(8bit per-tensor amax=2.1255 calibrator=HistogramCalibrator quant)
conv1.quantizer_w                       : TensorQuantizer(8bit per-tensor amax=0.1418 calibrator=HistogramCalibrator quant)
layer1.0.conv1.quantizer                : TensorQuantizer(8bit per-tensor amax=0.6120 calibrator=HistogramCalibrator quant)
layer1.0.conv1.quantizer_w              : TensorQuantizer(8bit per-tensor amax=0.0555 calibrator=HistogramCalibrator quant)
layer1.0.conv2.quantizer                : TensorQuantizer(8bit per-tensor amax=0.2950 calibrator=HistogramCalibrator quant)
layer1.0.conv2.quantizer_w              : TensorQuantizer(8bit per-tensor amax=0.0327 calibrator=HistogramCalibrator quant)
layer1.1.conv1.quantizer                : TensorQuantizer(8bit per-tensor amax=0.5320 calibrator=HistogramCalibrator quant)
layer1.1.conv1.quantizer_w              : TensorQuantizer(8bit per-tensor amax=0.0395 calibrator=HistogramCalibrator quant)
layer1.1

Epoch: [3]  [ 0/40]  eta: 0:29:41  lr: 0.0001  img/s: 2.878492948106378  loss: 0.1818 (0.1818)  acc1: 92.1875 (92.1875)  acc5: 100.0000 (100.0000)  time: 44.5340  data: 0.0662
Epoch: [3]  [ 1/40]  eta: 0:29:20  lr: 0.0001  img/s: 2.799503281035007  loss: 0.1600 (0.1709)  acc1: 92.1875 (93.7500)  acc5: 100.0000 (100.0000)  time: 45.1389  data: 0.0438
Epoch: [3]  [ 2/40]  eta: 0:28:43  lr: 0.0001  img/s: 2.7961661999991416  loss: 0.1600 (0.1601)  acc1: 94.5312 (94.0104)  acc5: 100.0000 (100.0000)  time: 45.3590  data: 0.0367
Epoch: [3]  [ 3/40]  eta: 0:28:45  lr: 0.0001  img/s: 2.5376503658803293  loss: 0.1384 (0.1533)  acc1: 94.5312 (94.9219)  acc5: 100.0000 (99.8047)  time: 46.6347  data: 0.0328
Epoch: [3]  [ 4/40]  eta: 0:28:23  lr: 0.0001  img/s: 2.5593578759479483  loss: 0.1600 (0.1586)  acc1: 95.3125 (95.0000)  acc5: 100.0000 (99.8438)  time: 47.3145  data: 0.0305
Epoch: [3]  [ 5/40]  eta: 0:27:29  lr: 0.0001  img/s: 2.768827542750056  loss: 0.1600 (0.1680)  acc1: 95.3125 (95.1823)

...

Epoch: [4]  [ 0/40]  eta: 0:29:06  lr: 0.0001  img/s: 2.9332588503465318  loss: 0.1356 (0.1356)  acc1: 93.7500 (93.7500)  acc5: 100.0000 (100.0000)  time: 43.6602  data: 0.0227
Epoch: [4]  [ 1/40]  eta: 0:28:34  lr: 0.0001  img/s: 2.8920512397326212  loss: 0.1356 (0.1652)  acc1: 93.7500 (93.7500)  acc5: 100.0000 (100.0000)  time: 43.9700  data: 0.0217
Epoch: [4]  [ 2/40]  eta: 0:27:59  lr: 0.0001  img/s: 2.8674970363942105  loss: 0.1356 (0.1473)  acc1: 93.7500 (94.5312)  acc5: 100.0000 (100.0000)  time: 44.1998  data: 0.0215
Epoch: [4]  [ 3/40]  eta: 0:27:24  lr: 0.0001  img/s: 2.836498196419857  loss: 0.1114 (0.1335)  acc1: 93.7500 (95.5078)  acc5: 100.0000 (100.0000)  time: 44.4414  data: 0.0261
Epoch: [4]  [ 4/40]  eta: 0:26:43  lr: 0.0001  img/s: 2.8515346620958413  loss: 0.1356 (0.1347)  acc1: 96.0938 (95.7812)  acc5: 100.0000 (100.0000)  time: 44.5349  data: 0.0251
Epoch: [4]  [ 5/40]  eta: 0:26:07  lr: 0.0001  img/s: 2.7804822544503565  loss: 0.1285 (0.1337)  acc1: 95.3125 (95.7

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [01:14<00:00, 37.37s/it]
W1117 22:55:23.306390 134165911848768 tensor_quantizer.py:173] Disable HistogramCalibrator
W1117 22:55:23.307151 134165911848768 tensor_quantizer.py:173] Disable HistogramCalibrator
W1117 22:55:23.308289 134165911848768 tensor_quantizer.py:173] Disable HistogramCalibrator
W1117 22:55:23.308887 134165911848768 tensor_quantizer.py:173] Disable HistogramCalibrator
W1117 22:55:23.309581 134165911848768 tensor_quantizer.py:173] Disable HistogramCalibrator
W1117 22:55:23.310149 134165911848768 tensor_quantizer.py:173] Disable HistogramCalibrator
W1117 22:55:23.310730 134165911848768 tensor_quantizer.py:173] Disable HistogramCalibrator
W1117 22:55:23.311317 134165911848768 tensor_quantizer.py:173] Disable HistogramCalibrator
W1117 22:55:23.311895 134165911848768 tensor_quantizer.py:173] Disable HistogramCalibrato

W1117 22:55:23.352282 134165911848768 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W1117 22:55:23.352877 134165911848768 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W1117 22:55:23.353487 134165911848768 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W1117 22:55:23.354093 134165911848768 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W1117 22:55:23.354695 134165911848768 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W1117 22:55:23.355286 134165911848768 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W1117 22:55:23.355890 134165911848768 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W1117 22:55:23.356482 134165911848768 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W1117 22:55:23.357075 134165911848768 tensor_quantizer.py:237] Load calibrated amax, shape=torch.Size([]).
W1117 22:55:23.357674 134165911848768

conv1.quantizer                         : TensorQuantizer(8bit per-tensor amax=2.1255 calibrator=HistogramCalibrator scale=59.752037048339844 quant)
conv1.quantizer_w                       : TensorQuantizer(8bit per-tensor amax=0.1418 calibrator=HistogramCalibrator scale=895.780029296875 quant)
layer1.0.conv1.quantizer                : TensorQuantizer(8bit per-tensor amax=0.5655 calibrator=HistogramCalibrator scale=216.89956665039062 quant)
layer1.0.conv1.quantizer_w              : TensorQuantizer(8bit per-tensor amax=0.0556 calibrator=HistogramCalibrator scale=2286.89501953125 quant)
layer1.0.conv2.quantizer                : TensorQuantizer(8bit per-tensor amax=0.2578 calibrator=HistogramCalibrator scale=466.114990234375 quant)
layer1.0.conv2.quantizer_w              : TensorQuantizer(8bit per-tensor amax=0.0327 calibrator=HistogramCalibrator scale=3880.2880859375 quant)
layer1.1.conv1.quantizer                : TensorQuantizer(8bit per-tensor amax=0.4872 calibrator=HistogramCalibrato

Epoch: [5]  [ 0/40]  eta: 0:28:06  lr: 0.0001  img/s: 3.0383591813009168  loss: 0.1451 (0.1451)  acc1: 96.8750 (96.8750)  acc5: 100.0000 (100.0000)  time: 42.1513  data: 0.0233
Epoch: [5]  [ 1/40]  eta: 0:28:09  lr: 0.0001  img/s: 2.8795351851226303  loss: 0.1282 (0.1367)  acc1: 96.0938 (96.4844)  acc5: 100.0000 (100.0000)  time: 43.3122  data: 0.0223
Epoch: [5]  [ 2/40]  eta: 0:27:49  lr: 0.0001  img/s: 2.8338368764406887  loss: 0.1282 (0.1288)  acc1: 96.0938 (96.3542)  acc5: 100.0000 (100.0000)  time: 43.9379  data: 0.0218
Epoch: [5]  [ 3/40]  eta: 0:27:16  lr: 0.0001  img/s: 2.8368380070865085  loss: 0.1130 (0.1123)  acc1: 96.0938 (96.8750)  acc5: 100.0000 (100.0000)  time: 44.2388  data: 0.0216
Epoch: [5]  [ 4/40]  eta: 0:26:35  lr: 0.0001  img/s: 2.8671100615712803  loss: 0.1130 (0.1095)  acc1: 96.8750 (96.8750)  acc5: 100.0000 (100.0000)  time: 44.3241  data: 0.0215
Epoch: [5]  [ 5/40]  eta: 0:25:51  lr: 0.0001  img/s: 2.885727337344602  loss: 0.0984 (0.1061)  acc1: 96.8750 (97.0

### Rerun model evaluation after re-training

In [ ]:
correct = 0
total = 0

model.eval()
start_time = timeit.default_timer()
with torch.no_grad():
    for iteraction, (images, labels) in tqdm(enumerate(data), total=len(data)):
        images, labels = images.to("cpu"), labels.to("cpu")
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(timeit.default_timer() - start_time)
print('Accuracy of the network on the 10000 test images: %.4f %%' % (
    100 * correct / total))

Accuracy after re-training: